In [1]:
import pandas as pd
from datetime import datetime, time

In [2]:
df = pd.read_csv("/content/market_data.csv")

In [3]:
df.head()

,Unnamed: 0,currentdate,currenttime,high,low,close,symbol
0,0,1/1/2024,9:15:00,654.35,648.35,654.35,AARTIIND
1,1,1/1/2024,9:20:00,654.75,652.10,653.00,AARTIIND
2,2,1/1/2024,9:25:00,653.35,651.00,652.80,AARTIIND
3,3,1/1/2024,9:30:00,652.95,651.10,652.45,AARTIIND
4,4,1/1/2024,9:35:00,654.00,652.50,653.75,AARTIIND


In [4]:
df.shape

(126117, 7)

In [5]:
df.isnull().sum()

,0
Unnamed: 0,0
currentdate,0
currenttime,0
high,0
low,0
close,0
symbol,0


In [6]:
# Convert to datetime
df['datetime'] = pd.to_datetime(df['currentdate'] + ' ' + df['currenttime'])
df['date'] = pd.to_datetime(df['currentdate']).dt.date
df = df.sort_values(['symbol', 'datetime'])

In [7]:
def backtest_orb_strategy(df):
    trades = []

    for symbol in df['symbol'].unique():
        symbol_df = df[df['symbol'] == symbol]

        for date in symbol_df['date'].unique():
            day_df = symbol_df[symbol_df['date'] == date].reset_index(drop=True)

            if len(day_df) < 2:
                continue

            first_candle = day_df.iloc[0]
            orb_high = first_candle['high']
            orb_low = first_candle['low']

            long_trade_placed = False
            short_trade_placed = False

            for i, row in day_df.iterrows():
                curr_time = row['datetime'].time()
                price = row['close']

                # Exit at 15:15
                if curr_time >= time(15, 15):
                    break

                # Long Entry
                if not long_trade_placed and price > orb_high:
                    entry_price = price
                    target = entry_price * 1.005
                    stop_loss = entry_price * 0.9975

                    # Monitor trade
                    for j in range(i+1, len(day_df)):
                        future_row = day_df.iloc[j]
                        future_price = future_row['close']
                        exit_time = future_row['datetime']

                        if future_price >= target:
                            trades.append([date, symbol, 'BUY', entry_price, future_price, row['datetime'], exit_time, 'Target Hit', round(future_price - entry_price, 2)])
                            break
                        elif future_price <= stop_loss:
                            trades.append([date, symbol, 'BUY', entry_price, future_price, row['datetime'], exit_time, 'Stop Loss', round(future_price - entry_price, 2)])
                            break
                        elif future_row['datetime'].time() >= time(15, 15):
                            trades.append([date, symbol, 'BUY', entry_price, future_price, row['datetime'], exit_time, 'Time Exit', round(future_price - entry_price, 2)])
                            break

                    long_trade_placed = True

                # Short Entry
                if not short_trade_placed and price < orb_low:
                    entry_price = price
                    target = entry_price * 0.995
                    stop_loss = entry_price * 1.0025

                    # Monitor trade
                    for j in range(i+1, len(day_df)):
                        future_row = day_df.iloc[j]
                        future_price = future_row['close']
                        exit_time = future_row['datetime']

                        if future_price <= target:
                            trades.append([date, symbol, 'SELL', entry_price, future_price, row['datetime'], exit_time, 'Target Hit', round(entry_price - future_price, 2)])
                            break
                        elif future_price >= stop_loss:
                            trades.append([date, symbol, 'SELL', entry_price, future_price, row['datetime'], exit_time, 'Stop Loss', round(entry_price - future_price, 2)])
                            break
                        elif future_row['datetime'].time() >= time(15, 15):
                            trades.append([date, symbol, 'SELL', entry_price, future_price, row['datetime'], exit_time, 'Time Exit', round(entry_price - future_price, 2)])
                            break

                    short_trade_placed = True

    return trades


In [8]:
# Run backtest
trade_log = backtest_orb_strategy(df)

# Convert to DataFrame
columns = ['Date', 'Symbol', 'Type', 'Entry Price', 'Exit Price', 'Entry Time', 'Exit Time', 'Exit Reason', 'PnL']
results_df = pd.DataFrame(trade_log, columns=columns)

In [9]:
# Trade Count per Day (start from 1 each day, sorted by Entry Time)
results_df['Trade Count'] = results_df.sort_values(['Date', 'Entry Time']).groupby('Date').cumcount() + 1

In [13]:
# Save to CSV
results_df.to_csv("orb_trade_log2.csv", index=False)

In [12]:
results_df

,Date,Symbol,Type,Entry Price,Exit Price,Entry Time,Exit Time,Exit Reason,PnL,Trade Count
0,2024-01-01,AARTIIND,BUY,655.05,653.00,2024-01-01 09:45:00,2024-01-01 09:55:00,Stop Loss,-2.05,18
1,2024-01-02,AARTIIND,BUY,658.10,655.55,2024-01-02 09:30:00,2024-01-02 09:50:00,Stop Loss,-2.55,16
2,2024-01-02,AARTIIND,SELL,651.70,646.80,2024-01-02 09:55:00,2024-01-02 10:25:00,Target Hit,4.90,23
3,2024-01-03,AARTIIND,SELL,647.90,649.75,2024-01-03 09:35:00,2024-01-03 09:50:00,Stop Loss,-1.85,12
4,2024-01-04,AARTIIND,SELL,649.85,652.30,2024-01-04 09:25:00,2024-01-04 09:40:00,Stop Loss,-2.45,7
...,...,...,...,...,...,...,...,...,...,...
2133,2024-03-26,IEX,BUY,136.25,135.50,2024-03-26 09:25:00,2024-03-26 09:30:00,Stop Loss,-0.75,11
2134,2024-03-26,IEX,SELL,134.90,134.15,2024-03-26 14:40:00,2024-03-26 15:10:00,Target Hit,0.75,30
2135,2024-03-27,IEX,SELL,134.20,134.55,2024-03-27 09:50:00,2024-03-27 10:30:00,Stop Loss,-0.35,18
2136,2024-03-27,IEX,BUY,135.40,135.00,2024-03-27 11:10:00,2024-03-27 11:20:00,Stop Loss,-0.40,30
